In [2]:
import numpy as np
import pandas as pd

In [38]:
import gzip
import struct

In [178]:
sys.byteorder

'little'

##### TRAINING SET IMAGE FILE (train-images-idx3-ubyte)
##### TRAINING SET LABEL FILE (train-labels-idx1-ubyte)
##### TEST SET LABEL FILE (t10k-labels-idx1-ubyte)
##### TEST SET IMAGE FILE (t10k-images-idx3-ubyte)

In [540]:
files = {'imagefile':'image files//train-images-idx3-ubyte.gz','imagelabel':'image files//train-labels-idx1-ubyte.gz'
        ,'testimage':'image files//t10k-images-idx3-ubyte.gz','testlabel':'image files//t10k-labels-idx1-ubyte.gz'}

In [361]:
f=gzip.open('image files//train-images-idx3-ubyte.gz','rb')

In [262]:
try:
    magic = struct.unpack('4B', f.read(4))
except struct.error as  e:
    print(e)

In [263]:
print (magic)

(0, 0, 8, 3)


In [264]:
try:
    f.seek(4)
    images = struct.unpack('>I',f.read(4))[0]
    rows = struct.unpack('>I',f.read(4))[0]
    columns = struct.unpack('>I',f.read(4))[0]
    print(images,rows,columns)
except struct.error as e:
    print(e)

60000 28 28


In [466]:
# getting the image data
start = f.seek(16)
pixel_im = []
for i in range(images):
    f.seek(start + (i * rows * columns))
    pixel = np.array(struct.unpack('>784B',f.read(rows * columns)))
    pixel_im.append(pixel)
pixel_im = np.array(pixel_im)

In [267]:
f = gzip.open('image files//train-labels-idx1-ubyte.gz','rb')

In [268]:
try:
    magic = struct.unpack('4B', f.read(4))
except struct.error as  e:
    print(e)

In [269]:
print(magic)

(0, 0, 8, 1)


In [270]:
try:
    f.seek(4)
    images = struct.unpack('>I',f.read(4))[0]
    print(images)
except struct.error as e:
    print(e)

60000


In [292]:
##loading the labels
try:
    start = f.seek(8)
    labels = np.array(struct.unpack('>60000B',f.read(images)))
except struct.error as e:
    print(e)

In [449]:
##Initialize the weights for the input layer
import random
def getRandomWeights(a,b):
    W = np.empty((0, 784), int)
    for j in range(10):
        w = np.array([random.uniform(a, b) for i in range(rows * columns)])
        W = np.vstack([w, W])               # weight vector Ω
    return W

In [467]:
W.shape

(10, 784)

In [385]:
def getLabel(labels,idx):
    if (idx <= images):
        return (labels[idx])

In [381]:
def signFunc(x):
    if (x < 0):
        return -1
    elif (x == 0):
        return 0
    elif (x > 0 ):
        return 1

In [421]:
def stepFunc(x):
    if (x >= 0):
        return 1
    else:
        return 0

In [430]:
#@param: array 
def signFunction(X):
    if (type(X) == np.ndarray or type(X) == list):
        for idx,x in enumerate(X):
            X[idx] = signFunc(x)
    elif (type(X) == int):
        X = signFunc(X)
    return X

In [431]:
#@param : aray,list or int
def stepFunction(X):
    if (type(X) == np.ndarray or type(X) == list):
        for idx,x in enumerate(X):
            X[idx] = stepFunc(x)
    elif (type(X) == int):
        X = stepFunc(x)
    return(X)

In [ ]:
mis = 0
for X_ele in S:
    X = np.array([1] + list(X_ele))  # X input vector
    y = self.signFunc(W1.T @ X)
    d = self.boolClass(X_ele)             #desired input
    if (y == 1 and d == 0):
        W1 = W1 - ( rate * X)
        mis += 1
    elif (y == 0 and d == 1):
        W1 = W1 + ( rate * X)
        mis += 1
return((W1,mis))

In [350]:
len(pixel_im[0].T)

784

In [386]:
def getDesiredInput(labels,idx):
    d = np.zeros(10)
    label = getLabel(labels,idx)
    d[label] = 1
    return (d)

In [468]:
np.array(pixel_im).shape[1]

784

In [514]:
print(np.argmax(W @ (pixel_im[0]) ),getLabel(labels,0))

1 5


In [515]:
W = getRandomWeights(-1,1)

In [559]:
# PTA for mnsit
rate = 1
e = 0.1
n = images
def PTA_mnist(n,rate,e):
    epoch = 0
    epoch_err = []
    while True:
        mis = 0
        for idx in range(n):
            v = W @ pixel_im[idx]
            if (np.argmax(v) != getLabel(labels,idx)):
                mis += 1
        epoch_err.append(mis)
        epoch = epoch + 1
        for idx in range(n):
            W = W + (getDesiredInput(labels,idx) - stepFunction(W @ pixel_im[idx])).reshape(-1,1) @ (pixel_im[idx].reshape(-1,1).T)
        print(epoch_err[epoch-1]/images)
        if (epoch_err[epoch-1]/images < e ):
            break
    return W

In [531]:
epoch

1

In [ ]:
W = PTA_mnist(n,rate,e)

In [542]:
#testing
# open file
f = gzip.open(files['testimage'])

In [551]:
try:
    f.seek(0)
    magic = struct.unpack('4B',f.read(4))
    print(magic)
except struct.error as e:
    print (e)

(0, 0, 8, 3)


In [591]:
def getImages(f):
    try:
        f.seek(4)
        images = struct.unpack('>I', f.read(4))[0]
        rows = struct.unpack('>I', f.read(4))[0]
        columns = struct.unpack('>I', f.read(4))[0]
        start = f.seek(16)
        size =  rows *columns
        pixel_im = []
        for i in range(images):
            f.seek(start + (i * size))
            pixel = np.array(struct.unpack('>'+'B'*size, f.read(size)))
            pixel_im.append(pixel)
        return (np.array(pixel_im),images,rows,columns)
    except struct.error as e:
        print(e)

In [574]:
#return labels of image from Mnist data
def getLabels(f):
    try:
        f.seek(4)
        images = struct.unpack('>I', f.read(4))[0]
        f.seek(8)
        labels = np.array(struct.unpack('>'+'B'*images, f.read(images)))
        return(labels)
    except struct.error as e:
        print(e)

In [570]:
f = gzip.open(files['testlabel'])
try:
    f.seek(0)
    magic = struct.unpack('4B', f.read(4))
    print(magic)
    f.seek(4)
    images = struct.unpack('>I',f.read(4))[0]
    labels = np.array(struct.unpack('>'+'B'*images,f.read(images)))
except struct.error as e:
    print(e)

(0, 0, 8, 1)


In [578]:
f = gzip.open(files['testlabel'])
testLabels = getLabels(f)
testLabels.shape

(10000,)

In [592]:
f = gzip.open(files['testimage'])
testImages,testNoImages,rows,cols = getImages(f)

In [581]:
def testing(W,testLabels,testImages,n):
    error = 0
    for idx in range(n):
        v = W @ testImages[idx]
        if (np.argmax(v) != testLabels[idx]):
              error +=1
    return error

In [589]:
n= (testNoImages)
testing(W,testLabels,testImages,n)

1219